# <p style="text-align: center;">ETL OUTPUT_STEAM_GAMES.</p>

PROCEDO A ABRIR Y ANALIZAR LOS ARCHIVOS JSON DE LOS CUALES VAMOS A TOMAR LA INFORMACION.

In [40]:
# Realizo carga de librerias.

import ast
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

##### OUTPUT_STEAM_GAMES

In [41]:
# Cargo arhivo JSON.

filas = []

with open(r"D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\ARCHIVOS JSON\output_steam_games.json", "r", encoding="Latin-1") as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            filas.append(objeto_json)
        except json.JSONDecodeError:
            print(f"Error de formato JSON en la línea: {linea}")

#Convierto la lista de objetos JSON en un DataFrame
df_games = pd.DataFrame(filas)

In [42]:
# Estudio el detalle de los campos.
df_games.info()

# Veo el 'Head'
df_games

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


BUSCO Y NORMALIZO NULOS.

Valores Nulos de cada columna

In [43]:
nulos1 = df_games.isna().sum()
nulos1

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [44]:
# Cuento filas completamente nulas.
filas_completamente_nulas = df_games.isnull().all(axis=1).sum()
print("Número de registros completamente nulos:")
print(filas_completamente_nulas)

Número de registros completamente nulos:
88310


In [45]:
# Verifico si hay algún valor distinto de None en la columna 'A'
col_a_has_not_none = df_games['genres'].notna().any()
print(col_a_has_not_none)  # Esto imprimirá True si hay algún valor que no sea None

True


### Analizo y normalizo valores anidados.

In [49]:
def verificar_columna(column):
    tiene_anidados = any(isinstance(valor, (list, dict, tuple,set)) for valor in column)
    tipos_datos = set(type(valor).__name__ for valor in column)
    return tiene_anidados, tipos_datos

# Verifico cada columna especificada
for col in df_games:
    tiene_anidados, tipos_datos = verificar_columna(df_games[col])
    if tiene_anidados:
        print(f'La columna "{col}" tiene datos anidados')
        print(f'Tipos de datos en la columna "{col}": {tipos_datos}')
    else:
        print(f'La columna "{col}" no tiene datos anidados')
        print(f'Tipos de datos en la columna "{col}": {tipos_datos}')
      
    

La columna "genres" tiene datos anidados
Tipos de datos en la columna "genres": {'list', 'float'}
La columna "app_name" no tiene datos anidados
Tipos de datos en la columna "app_name": {'float', 'str'}
La columna "release_date" no tiene datos anidados
Tipos de datos en la columna "release_date": {'float', 'str'}
La columna "price" no tiene datos anidados
Tipos de datos en la columna "price": {'float', 'str'}
La columna "id" no tiene datos anidados
Tipos de datos en la columna "id": {'float', 'str'}
La columna "developer" no tiene datos anidados
Tipos de datos en la columna "developer": {'float', 'str'}


### Elimino campos irrelevantes para la confeccion del proyecto.

In [ ]:
df_games.drop(['publisher', 'url', 'tags', 'reviews_url', 'specs', 'early_access', 'title'], axis=1, inplace=True)
df_games

#### Modifico el nombre del campo 'id' a 'item_id' para compatibilizar con otras tablas.

In [53]:
df_games.rename(columns={'id': 'item_id'}, inplace=True)
df_games

,genres,app_name,release_date,price,item_id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns"


In [54]:
#Checueo nuevamente como quedo el Dataframe.
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28852 non-null  object
 1   app_name      32133 non-null  object
 2   release_date  30068 non-null  object
 3   price         30758 non-null  object
 4   item_id       32133 non-null  object
 5   developer     28836 non-null  object
dtypes: object(6)
memory usage: 5.5+ MB


#### Trabajo sobre la columna 'genres' y aplico dummies para poder trabajar con als categorias de cada producto.

In [55]:
df_games['genres'] = df_games['genres'].fillna('[]')  # Relleno valores faltantes con una lista vacía
df_games['genres'] = df_games['genres'].apply(lambda x: ', '.join(x))  # Convierto la lista de géneros a una cadena separada por comas

#Creo variables ficticias para los géneros
dummy_genres = df_games['genres'].str.get_dummies(', ')

#Concateno las variables ficticias con el DataFrame original
df_games = pd.concat([df_games, dummy_genres], axis=1)
df_games

,genres,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,]
0,"[, ]",NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,"[, ]",NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,"[, ]",NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,"[, ]",NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,"[, ]",NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,"Casual, Indie, Simulation, Strategy",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS""",0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
120442,"Indie, Racing, Simulation",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
120443,"Casual, Indie",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Depuro registros a fin de simplificar e proceso.

In [56]:
# Eliminamos filas que tienen nulos en todas las columnas especificadas
df_games_cleaned = df_games.dropna(subset=['app_name', 'release_date', 'price', 'item_id', 'developer'], how='all')

# Verificamos los primeros registros para asegurarnos de que el proceso ha funcionado
df_games_cleaned



,genres,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,]
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
88311,"Free to Play, Indie, RPG, Strategy",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
88312,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
88313,"Action, Adventure, Casual",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
88314,"[, ]",Log Challenge,NaN,2.99,773570,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,"Casual, Indie, Simulation, Strategy",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS""",0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
120442,"Indie, Racing, Simulation",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
120443,"Casual, Indie",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Modifico el tipo de dato en la columna 'release_date' y genero un nuevo campo 'year'.

In [58]:
df_games_cleaned["release_date"] = pd.to_datetime(df_games_cleaned["release_date"], errors="coerce")
df_games_cleaned["year"] = pd.to_datetime(df_games_cleaned["release_date"]).dt.year


df_games_cleaned["year"] = df_games_cleaned["year"].fillna(0)
df_games_cleaned["year"] = df_games_cleaned["year"].astype(int)

df_games_cleaned.tail(4)

C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_11600\3644462051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_cleaned["release_date"] = pd.to_datetime(df_games_cleaned["release_date"], errors="coerce")
C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_11600\3644462051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_cleaned["year"] = pd.to_datetime(df_games_cleaned["release_date"]).dt.year
C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_11600\3644462051.py:5: SettingWithCopyWarning: 
A v

,genres,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,...,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,],year
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2018
120442,"Indie, Racing, Simulation",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2018
120443,"Casual, Indie",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns",0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017
120444,"[, ]",Maze Run VR,NaT,4.99,681550,NaN,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


##### Elimino duplicados.

##### Trabajo sobre el campo 'price' a fin de depurar datos que complicarian el analisis.

In [59]:
# Supongamos que 'df' es tu DataFrame

# Filtrar solo los valores de tipo string en la columna 'price'
valores_string = df_games_cleaned.loc[df_games_cleaned['price'].apply(lambda x: isinstance(x, str)), 'price']

# Encontrar los valores únicos
valores_unicos_string = valores_string.unique()

# Mostrar los valores únicos
print(valores_unicos_string)

['Free To Play' 'Free to Play' 'Free' 'Free Demo' 'Play for Free!'
 'Install Now' 'Play WARMACHINE: Tactics Demo' 'Free Mod' 'Install Theme'
 'Third-party' 'Play Now' 'Free HITMAN™ Holiday Pack' 'Play the Demo'
 'Starting at $499.00' 'Starting at $449.00' 'Free to Try' 'Free Movie'
 'Free to Use']


Normalizo el campo 'Price'

In [60]:
# Reemplazar los valores específicos por 'Free' en la columna 'price' del DataFrame df_games_cleaned
df_games_cleaned['price'] = df_games_cleaned['price'].replace(['Free To Play', 'Free to Try', 'Free Mod', 'Free Demo', 'Play for Free!','Free to Play','Free to Use','Free Movie'], 'Free')

df_games_cleaned['price'] = df_games_cleaned['price'].replace(['Starting at $499.00', 'Starting at $449.00'], 499)

df_games_cleaned['price'] = df_games_cleaned['price'].replace('Free', 0)

df_games_cleaned = df_games_cleaned[pd.to_numeric(df_games_cleaned['price'], errors='coerce').notnull()]

# Verificar los cambios
print(df_games_cleaned['price'].unique())


[4.99 0 0.99 2.99 3.99 9.99 18.99 29.99 10.99 1.59 14.99 1.99 59.99 8.99
 6.99 7.99 39.99 19.99 7.49 12.99 5.99 2.49 15.99 1.25 24.99 17.99 61.99
 3.49 11.99 13.99 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 21.99 89.99 0.98
 139.92 4.29 64.99 54.99 74.99 0.89 0.5 299.99 1.29 3.0 15.0 5.49 23.99
 49.0 20.99 10.93 1.39 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5 199.0 189.0
 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0 40.0 3.33
 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95 499.99
 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97 10.96 10.0
 30.0 2.66 6.48 19.29 11.15 18.9 2.89 99.0 87.94 599.0 8.98 9.69 0.49 9.98
 9.95 7.0 499 12.89 6.49 1.87 42.99 41.99 289.99 23.96 5.65 12.0 13.37
 189.96 124.99 19.98 160.91]


C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_11600\3723190115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_cleaned['price'] = df_games_cleaned['price'].replace(['Free To Play', 'Free to Try', 'Free Mod', 'Free Demo', 'Play for Free!','Free to Play','Free to Use','Free Movie'], 'Free')
C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_11600\3723190115.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_cleaned['price'] = df_games_cleaned['price'].replace(['Starting at $499.00', 'Startin

##### Reseteo los indices.

In [61]:
df_games_cleaned.reset_index(drop=True, inplace=True)

In [62]:
df_games_cleaned

,genres,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,...,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,],year
0,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,0,1,0,0,...,1,0,0,1,0,0,0,0,0,2018
1,"Free to Play, Indie, RPG, Strategy",Ironbound,2018-01-04,0,643980,Secret Level SRL,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2018
2,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017-07-24,0,670290,Poolians.com,0,0,0,0,...,1,0,1,0,0,0,0,0,0,2017
3,"Action, Adventure, Casual",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2017
4,"[, ]",Log Challenge,NaT,2.99,773570,NaN,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30744,"Casual, Indie, Simulation, Strategy",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS""",0,0,0,0,...,1,0,0,1,0,0,0,0,0,2018
30745,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2018
30746,"Indie, Racing, Simulation",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2018
30747,"Casual, Indie",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns",0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017


##### Procedo a guardar el dataframe en formato CSV y PARQUET.

In [63]:
df_games_cleaned.to_csv(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_games.csv', index=False)

In [64]:
df_games_cleaned.to_parquet(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\DATAFRAMES PARKET\df_games.parquet')

c:\Users\Arnaldo\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
